# ***Credit Card Fraud Detection using Logistic Regression***

### Objective:

This machine learning project uses anonymized credit card transaction data to predict whether a transaction is fraudulent or legitimate, helping financial institutions reduce losses from fraud.

---

### Workflow Overview:

* ***Import Libraries*** – Load necessary Python packages
* ***Load & Explore Data*** – Understand the structure, class imbalance, and statistical properties
* ***Handle Class Imbalance*** – Apply under-sampling to create a balanced dataset
* ***Feature & Target Split*** – Separate input features (X) and the target label (Y)
* ***Train-Test Split*** – Split the dataset for training and evaluation
* ***Model Training*** – Train a Logistic Regression classifier
* ***Evaluate Model*** – Use accuracy, confusion matrix, and classification report
* ***Interpret Metrics*** – Gain insights from precision, recall, and F1-score

---

### Problem Type:

* Binary Classification (0 = Legitimate, 1 = Fraudulent)

---

### Dataset:

* **Source:** [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)
* **Rows:** 284,807 transactions
* **Features:**

  * `Time`: Time elapsed since the first transaction
  * `V1` to `V28`: PCA-transformed features to anonymize data
  * `Amount`: Transaction value in Euros
  * `Class`:

    * `0` → Legitimate
    * `1` → Fraudulent

---




# Step - 1
### Importing Dependencies
We'll use `NumPy`, `Pandas` for data handling, and `Scikit-learn` for model building and evaluation.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step - 2
### Data Collection and Preprocessing

In [3]:
# Load a CSV file directly from a ZIP archive into a DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv.zip', compression='zip')

In [4]:
# Print first few rows of data
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


- ***Time***: Seconds elapsed between this transaction and the first transaction in the dataset

- ***V1 to V28***: Principal components obtained from PCA (to protect confidentiality); these are numerical and standardized

- ***Amount*** : The transaction amount (in Euros)

- ***Class***: The target variable —
  - 0 = Legitimate transaction
  - 1 = Fraudulent transaction

In [5]:
# Print the dimension of our dataset (rows,columns)
credit_card_data.shape

(284807, 31)

In [25]:
# Display basic info including data types and missing values
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [26]:
# Check for missing values in each column
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [27]:
# Count of legit vs fraudulent transactions
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


***Observation***
- This dataset is highly imbalanced — legit transactions far outnumber fraudulent ones.
- Special handling (like under-sampling or over-sampling) is required before training a model.



***Seprating data for further analysis***

In [28]:
# Extract all legitimate transactions
legit = credit_card_data[credit_card_data.Class == 0 ]

In [29]:
# Extract all fraudulent transactions
fraud = credit_card_data[credit_card_data.Class == 1 ]

In [30]:
# Print the shape of both subsets
print(f"Shape of legit data:- {legit.shape}")
print(f"Shape of fraud data:- {fraud.shape}")

Shape of legit data:- (284315, 31)
Shape of fraud data:- (492, 31)


In [31]:
# Statistical summary of transaction amounts
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [19]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [32]:
# Compare mean values of features across both classes
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


***Inference***
- The average values for several features (especially `Amount`) differ significantly between legit and fraudulent transactions.
- These differences help our model learn patterns to detect fraud effectively.

# Step - 3
### Data Analysis

***UNDER-SAMPLING:***
- Create a balanced dataset by randomly selecting 492 legitimate transactions to match the number of fraud cases


In [33]:
legit_sample = legit.sample(n = 492)    # n = number of datapoints you want to have (492 here)

In [34]:
# Combine the sampled legit and all fraud transactions into a new balanced dataset
new_dataset = pd.concat([legit_sample, fraud] , axis = 0)   # axis = 0 means df will be added row wise

In [35]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
74398,55560.0,1.022768,-0.575025,0.005209,-1.510317,-0.171678,0.313211,-0.249063,0.262787,1.200388,-1.228802,1.122107,0.902732,-0.449040,0.585995,2.818584,-1.857606,1.206200,-2.178784,-0.955942,-0.148548,0.123766,0.489472,0.072647,-0.599439,0.159096,0.062809,0.056335,0.009146,56.00,0
209634,137642.0,2.018761,0.138112,-1.596532,0.342355,0.413838,-0.654975,0.111240,-0.121912,0.167112,-0.220226,1.359605,1.087962,0.305722,-0.707945,-0.548284,0.441398,0.270338,0.082583,0.269305,-0.127513,-0.296348,-0.768416,0.346022,0.679634,-0.301158,0.141350,-0.063965,-0.037921,1.98,0
146810,87899.0,-1.240200,1.219700,-0.086411,-2.973820,0.895859,-0.492567,1.065555,0.130174,0.623186,-0.104066,0.095542,-0.166756,-1.486611,0.377489,-0.911818,0.665707,-1.227407,-0.002394,-0.363380,0.257001,-0.381915,-0.980396,-0.122666,0.048447,0.092108,0.375353,0.525643,0.334852,5.00,0
245269,152674.0,1.910163,-0.157871,0.215995,1.366330,-0.710781,-0.209433,-0.650153,0.001146,0.972148,0.102323,-0.713444,1.075982,1.444326,-0.465237,0.568808,0.455039,-0.795836,0.015562,-0.758517,-0.160205,0.077581,0.451199,0.265668,-0.022480,-0.314331,-0.757778,0.088776,-0.016769,11.40,0
75243,55967.0,-4.611536,-3.045197,1.030047,0.589611,-0.852118,0.721861,0.974295,-0.374813,0.846436,-0.354624,1.098729,1.045668,-0.024894,-0.691313,-0.878922,0.429819,-0.298801,-0.633147,-0.185688,-1.977591,-0.761778,-0.390963,-2.785320,0.379677,-0.432542,0.259524,0.696922,-2.002774,557.76,0


In [36]:
# Check if the new dataset preserves the original characteristics
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,95146.276423,0.139250,-0.024160,-0.045708,-0.001539,-0.053500,0.028614,0.105599,-0.050917,-0.083693,0.058697,-0.034706,0.049630,-0.018471,-0.028159,-0.013697,-0.053732,0.000585,-0.002374,-0.076214,0.018873,-0.022865,-0.037057,-0.005538,0.030759,-0.007356,0.035894,0.005628,-0.007472,104.543557
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


***Note***
- The mean values in the new dataset are very similar to those in the original, which suggests the nature of our dataset is not changed.
- This particular step help to check if we got a good or bad sample; if we got a bad sample then the mean value is going to be very diffrent from previous dataframe which could mislead the model.

# Step - 4
### Feature and Target Split

In [37]:
# Features
X = new_dataset.drop(columns = 'Class' , axis = 1)

In [38]:
# Target
Y = new_dataset['Class']

In [39]:
# Preview Features
print(X)

            Time        V1        V2  ...       V27       V28  Amount
74398    55560.0  1.022768 -0.575025  ...  0.056335  0.009146   56.00
209634  137642.0  2.018761  0.138112  ... -0.063965 -0.037921    1.98
146810   87899.0 -1.240200  1.219700  ...  0.525643  0.334852    5.00
245269  152674.0  1.910163 -0.157871  ...  0.088776 -0.016769   11.40
75243    55967.0 -4.611536 -3.045197  ...  0.696922 -2.002774  557.76
...          ...       ...       ...  ...       ...       ...     ...
279863  169142.0 -1.927883  1.125653  ...  0.292680  0.147968  390.00
280143  169347.0  1.378559  1.289381  ...  0.389152  0.186637    0.76
280149  169351.0 -0.676143  1.126366  ...  0.385107  0.194361   77.89
281144  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700  245.00
281674  170348.0  1.991976  0.158476  ...  0.002988 -0.015309   42.53

[984 rows x 30 columns]


In [40]:
# Preview Target
print(Y)

74398     0
209634    0
146810    0
245269    0
75243     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


# Step - 5
### Train Test Split

- Split the data into training and testing sets (80% train, 20% test)
- Using stratify=Y to maintain the class distribution in both sets

In [41]:
X_train , X_test , Y_train , Y_test = train_test_split(
    X, Y , test_size= 0.2 , random_state=2 , stratify= Y
)

In [43]:
# Display shape of the datasets
print(f"Feature data shape: {X.shape}")
print(f"X_train data shape: {X_train.shape}")
print(f"X_test data shape: {X_test.shape}")

Feature data shape: (984, 30)
X_train data shape: (787, 30)
X_test data shape: (197, 30)


# Step - 6
### Model Training

In [44]:
# Load an instance of the Logistic Regression model
model = LogisticRegression()

In [45]:
# train data on training data
model.fit(X_train, Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Step - 7
### Model Evaluation

In [49]:
# Predict outcomes on the training data
train_pred = model.predict(X_train)

In [50]:
# Calculate accuracy score on training data
training_data_accuracy = accuracy_score(Y_train , train_pred)

In [51]:
# Print the training accuracy
print(f"Accuracy score of Training data:- {training_data_accuracy}")

Accuracy score of Training data:- 0.9440914866581956


In [52]:
# Predict outcomes on the test data
test_pred = model.predict(X_test)

In [53]:
# Calculate accuracy score on test data
test_data_accuracy = accuracy_score(Y_test , test_pred)

In [54]:
# Print the test accuracy
print(f"Accuracy score of Test data:- {test_data_accuracy}")

Accuracy score of Test data:- 0.9441624365482234


In [56]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict on test data
test_pred = model.predict(X_test)

# Confusion Matrix
conf_matrix = confusion_matrix(Y_test, test_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
print("\nClassification Report:")
print(classification_report(Y_test, test_pred, target_names=["Legit", "Fraud"]))

Confusion Matrix:
[[95  4]
 [ 7 91]]

Classification Report:
              precision    recall  f1-score   support

       Legit       0.93      0.96      0.95        99
       Fraud       0.96      0.93      0.94        98

    accuracy                           0.94       197
   macro avg       0.94      0.94      0.94       197
weighted avg       0.94      0.94      0.94       197



**Evaluation Summary**

- **Confusion Matrix**

```
               Predicted
             | Legit | Fraud
  -----------|-------|-------
  Actual Legit |  95  |   4
  Actual Fraud |   7  |  91
```

* **True Positives (TP)**: 91 → Correctly predicted fraud
  * **True Negatives (TN)**: 95 → Correctly predicted legit
  * **False Positives (FP)**: 4 → Legit predicted as fraud
  * **False Negatives (FN)**: 7 → Fraud predicted as legit

---

- **Classification Report Insights**

| Metric    | Legit | Fraud |
| --------- | ----- | ----- |
| Precision | 0.93  | 0.96  |
| Recall    | 0.96  | 0.93  |
| F1-score  | 0.95  | 0.94  |

#### Accuracy: `94%`

* The model correctly predicted 94% of all transactions.

#### Precision (Fraud): `0.96`

* Out of all transactions predicted as fraud, 96% were actually fraudulent.

#### Recall (Fraud): `0.93`

* Out of all actual frauds, the model identified 93%.

#### F1-score:

* A balanced score between precision and recall — very strong for both classes.

